# Zillow Clustering Project

#### Robert Murphy

#### June 22 2021
_________________________________

------------------------------------

# Imports:

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import TweedieRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import acquire
import explore
import zillow_wrangle
import prepare

In [2]:
# using a function which contains a implmentied sql query to bring in th dataset.
zillow_df = acquire.get_zillow_cached()
zillow_df= zillow_df.loc[:, ~zillow_df.columns.duplicated()]

zillow_df.head(1)

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,propertylandusedesc,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,storydesc,typeconstructiondesc
0,1727539,14297519,nan,nan,nan,3.50,4.00,None,nan,3.50,...,"60,590,630,072,012.00",0.03,2017-01-01,Single Family Residential,None,None,None,None,None,None


In [3]:
zillow_df.set_index('parcelid', inplace = True)

In [4]:
# 52,968 records x 67 fields
zillow_df.shape

(52953, 67)

In [5]:
zillow_df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,"52,953.00","1,497,097.55","859,452.25",349.00,"757,502.00","1,500,314.00","2,241,055.00","2,982,270.00"
airconditioningtypeid,"13,650.00",2.44,3.84,1.00,1.00,1.00,1.00,13.00
architecturalstyletypeid,70.00,7.10,2.67,2.00,7.00,7.00,7.00,21.00
basementsqft,50.00,679.72,689.70,38.00,273.00,515.00,796.50,"3,560.00"
bathroomcnt,"52,953.00",2.29,1.02,0.00,2.00,2.00,3.00,18.00
bedroomcnt,"52,953.00",3.29,0.96,0.00,3.00,3.00,4.00,14.00
buildingqualitytypeid,"33,807.00",6.27,1.72,1.00,5.00,6.00,8.00,12.00
calculatedbathnbr,"52,748.00",2.30,1.02,1.00,2.00,2.00,3.00,18.00
decktypeid,415.00,66.00,0.00,66.00,66.00,66.00,66.00,66.00
finishedfloor1squarefeet,"4,724.00","1,508.89",670.26,66.00,"1,112.00","1,383.00","1,718.00","6,912.00"


In [6]:
# looking into duplicates and missing values
# right away there is several columns which contain a high number of missing values.
#  
prepare.miss_dup_values(zillow_df)

Your selected dataframe has 67 columns.
There are 52 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values
buildingclasstypeid,52953,100.00
buildingclassdesc,52953,100.00
finishedsquarefeet15,52953,100.00
finishedsquarefeet13,52911,99.90
basementsqft,52903,99.90
storydesc,52903,99.90
storytypeid,52903,99.90
yardbuildingsqft26,52888,99.90
architecturalstyletypeid,52883,99.90
architecturalstyledesc,52883,99.90


In [7]:
prepare.summarize(zillow_df)

Dataframe head: 
               id  airconditioningtypeid  architecturalstyletypeid  \
parcelid                                                             
14297519  1727539                    nan                       nan   
17052889  1387261                    nan                       nan   
14186244    11677                    nan                       nan   
12177905  2288172                    nan                       nan   
12095076   781532                   1.00                       nan   
12069064   870991                    nan                       nan   
12790562  1246926                    nan                       nan   
11104527  1639362                   1.00                       nan   
13944538   249412                  13.00                       nan   
17110996    43675                    nan                       nan   

                 basementsqft          bathroomcnt           bedroomcnt  \
parcelid                                                           

---------------------

### Main Takeaways:

- Utilized a function which contains sql query to bring in dataset.
- Right away can see several columns which are missing several values over 50%. It might be ideal to just drop these columns.
- Will probably just keep columns which contain 90% non null values.
- Three datatypes; integer, float and object. 
- Some of the fields can be changed to a categorical utilizing 0 and 1.

--------------


# Prepare:

In [8]:
### No need to drop duplicates since there currently is none.###

In [9]:
# utilizing function which drops columns and rows that do not contain <90% non null values.
# this resulted in the shape ofd the df being 52801 records x 28 fields
zillow_check = prepare.handle_missing_values(zillow_df)
zillow_df = prepare.handle_missing_values(zillow_df)
zillow_check.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52801 entries, 14297519 to 12826780
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            52801 non-null  int64  
 1   bathroomcnt                   52801 non-null  float64
 2   bedroomcnt                    52801 non-null  float64
 3   calculatedbathnbr             52731 non-null  float64
 4   calculatedfinishedsquarefeet  52794 non-null  float64
 5   finishedsquarefeet12          52597 non-null  float64
 6   fips                          52801 non-null  float64
 7   fullbathcnt                   52731 non-null  float64
 8   latitude                      52801 non-null  float64
 9   longitude                     52801 non-null  float64
 10  lotsizesquarefeet             52423 non-null  float64
 11  propertycountylandusecode     52801 non-null  object 
 12  propertylandusetypeid         52801 non-null  floa

In [11]:
#dropping nan values
zillow_df.dropna(inplace = True )

In [14]:
# shape of dataframe is now 51059 records, 28 fields
zillow_df.shape

(51059, 28)

In [15]:
#To Do:
#Drops: propertylandusedesc,censustractandblock, assessmentyear,id, regionidcity
#Renames:  finishedsquarefeet12 rename to finished living area. Change fips to the county names. 
# Create columns for age of the house and drop year built.

In [16]:
zillow_df = prepare.remove_columns(zillow_df,['propertylandusedesc','censustractandblock', 'assessmentyear','id', 'regionidcity', 'roomcnt','propertylandusetypeid','calculatedbathnbr'])

In [17]:
# renaming two columns
zillow_df.rename(columns = {'finishedsquarefeet12':'finished_living_area', 'calculatedfinishedsquarefeet':'total_square_ft'}, inplace = True)

In [18]:
zillow_df.head(1).T

parcelid,14297519
bathroomcnt,3.50
bedroomcnt,4.00
total_square_ft,"3,100.00"
finished_living_area,"3,100.00"
fips,"6,059.00"
fullbathcnt,3.00
latitude,"33,634,931.00"
longitude,"-117,869,207.00"
lotsizesquarefeet,"4,506.00"
propertycountylandusecode,122


In [19]:
def get_counties(df):
    '''
    This function will create dummy variables out of the original fips column. 
    And return a dataframe with all of the original columns except regionidcounty.
    We will keep fips column for data validation after making changes. 
    New columns added will be 'LA', 'Orange', and 'Ventura' which are boolean 
    The fips ids are renamed to be the name of the county each represents. 
    '''
    # create dummy vars of fips id
    county_df = pd.get_dummies(df.fips)
    # rename columns by actual county name
    county_df.columns = ['LA', 'Orange', 'Ventura']
    # concatenate the dataframe with the 3 county columns to the original dataframe
    df_dummies = pd.concat([df, county_df], axis = 1)
    # drop regionidcounty and fips columns
    df_dummies = df_dummies.drop(columns = ['regionidcounty'])
    return df_dummies

In [20]:
get_counties(zillow_df)

,bathroomcnt,bedroomcnt,total_square_ft,finished_living_area,fips,fullbathcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,...,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,logerror,transactiondate,LA,Orange,Ventura
parcelid,,,,,,,,,,,,,,,,,,,,,
14297519,3.50,4.00,"3,100.00","3,100.00","6,059.00",3.00,"33,634,931.00","-117,869,207.00","4,506.00",122,...,"1,998.00","485,713.00","1,023,282.00","537,569.00","11,013.72",0.03,2017-01-01,0,1,0
17052889,1.00,2.00,"1,465.00","1,465.00","6,111.00",1.00,"34,449,266.00","-119,281,531.00","12,647.00",1110,...,"1,967.00","88,000.00","464,000.00","376,000.00","5,672.48",0.06,2017-01-01,0,0,1
14186244,2.00,3.00,"1,243.00","1,243.00","6,059.00",2.00,"33,886,168.00","-117,823,170.00","8,432.00",122,...,"1,962.00","85,289.00","564,778.00","479,489.00","6,488.30",0.01,2017-01-01,0,1,0
12177905,3.00,4.00,"2,376.00","2,376.00","6,037.00",3.00,"34,245,180.00","-118,240,722.00","13,038.00",0101,...,"1,970.00","108,918.00","145,143.00","36,225.00","1,777.51",-0.10,2017-01-01,1,0,0
12095076,3.00,4.00,"2,962.00","2,962.00","6,037.00",3.00,"34,145,202.00","-118,179,824.00","63,000.00",0101,...,"1,950.00","276,684.00","773,303.00","496,619.00","9,516.26",-0.00,2017-01-01,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12412492,2.00,4.00,"1,633.00","1,633.00","6,037.00",2.00,"33,870,815.00","-118,070,858.00","4,630.00",0100,...,"1,962.00","125,466.00","346,534.00","221,068.00","4,175.08",0.00,2017-09-19,1,0,0
11000655,2.00,2.00,"1,286.00","1,286.00","6,037.00",2.00,"34,245,368.00","-118,282,383.00","47,405.00",0100,...,"1,940.00","70,917.00","354,621.00","283,704.00","4,478.43",0.02,2017-09-20,1,0,0
17239384,2.00,4.00,"1,612.00","1,612.00","6,111.00",2.00,"34,300,140.00","-118,706,327.00","12,105.00",1111,...,"1,964.00","50,683.00","67,205.00","16,522.00","1,107.48",0.01,2017-09-21,0,0,1


In [21]:
# change fips to counties, create new columns for age (2017-yearbuilt), can change sqft to acerage

# create taxrate variable
zillow_df['taxrate'] = zillow_df.taxamount/zillow_df.taxvaluedollarcnt*100
# create acreage variable
zillow_df['acres'] = zillow_df.lotsizesquarefeet/43560

zillow_df['structure_dollar_per_sqft'] = zillow_df.structuretaxvaluedollarcnt/zillow_df.total_square_ft

zillow_df['land_dollar_per_sqft'] = zillow_df.landtaxvaluedollarcnt/zillow_df.lotsizesquarefeet

zillow_df['county_name'] = zillow_df['fips'].map({6037:'Los_Angeles', 6059:'Orange', 6111:'Ventura'})

In [22]:
zillow_df.head().T

parcelid,14297519,17052889,14186244,12177905,12095076
bathroomcnt,3.50,1.00,2.00,3.00,3.00
bedroomcnt,4.00,2.00,3.00,4.00,4.00
total_square_ft,"3,100.00","1,465.00","1,243.00","2,376.00","2,962.00"
finished_living_area,"3,100.00","1,465.00","1,243.00","2,376.00","2,962.00"
fips,"6,059.00","6,111.00","6,059.00","6,037.00","6,037.00"
fullbathcnt,3.00,1.00,2.00,3.00,3.00
latitude,"33,634,931.00","34,449,266.00","33,886,168.00","34,245,180.00","34,145,202.00"
longitude,"-117,869,207.00","-119,281,531.00","-117,823,170.00","-118,240,722.00","-118,179,824.00"
lotsizesquarefeet,"4,506.00","12,647.00","8,432.00","13,038.00","63,000.00"
propertycountylandusecode,122,1110,122,0101,0101


In [23]:
train, validate, split = prepare.split_continuous(zillow_df)

train -> (28592, 25)
validate -> (12255, 25)
test -> (10212, 25)


Main Takeaways:
- Removed Nan values and dropped columns with <90% non null values. Resulted in the shape of DF becoming 
-
-
-
